In [ ]:
"""
Revised file was created on Sun Dec 30 15:22 2024
@author: Nakyung Lee, University of Houston

Original file was created on Fri May 8 22:00:41 2020
@author: Ya Zhuo, University of Houston
"""

### Debye Temp Descriptor

In [45]:
import warnings
import numpy as np
import pandas as pd
from pymatgen.core.composition import Composition
from pymatgen.analysis.structure_analyzer import SpacegroupAnalyzer
from tqdm import tqdm
from mp_api.client import MPRester

In [46]:
def _custom_formatwarning(msg, *args, **kwargs):
    return f"{msg}\n"
warnings.formatwarning = _custom_formatwarning

polar_pointgroups = ["1","2","m","mm2","3","3m","4","4m","6","6mm"]
SG_data = pd.read_excel(r"SG to xtal class.xlsx") #Check the file name

def get_struc_dict(structure):
    struc_dict = {}
    sga = SpacegroupAnalyzer(structure)
    conv_struc = sga.get_conventional_standard_structure()
    space_group_number = sga.get_space_group_number()
    struc_dict["Space Group"] = space_group_number
    
    matching_row = SG_data[SG_data["Space Group"] == space_group_number]
    
    if not matching_row.empty:
        struc_dict["Crystal System"] = matching_row["Crystal System"].values[0]
        struc_dict["Laue"] = matching_row["Laue"].values[0]
        struc_dict["Crystal Class"] = matching_row["Crystal Class"].values[0]
    else:
        struc_dict["Crystal System"] = "Not found"
        struc_dict["Laue"] = "Not found"
        struc_dict["Crystal Class"] = "Not found"

    struc_dict["inversion center"] = sga.is_laue()
    struc_dict["inversion center"] = int(struc_dict["inversion center"])
    struc_dict["polar axis"] = sga.get_point_group_symbol() in polar_pointgroups
    struc_dict["polar axis"] = int(struc_dict["polar axis"])
    struc_dict["Reduced Volume"] = (conv_struc.volume / conv_struc.composition.get_reduced_composition_and_factor()[1])
    struc_dict["density"] = conv_struc.density
    struc_dict["avg anisotropy"] = ((conv_struc.lattice.a / conv_struc.lattice.b)+(conv_struc.lattice.b / conv_struc.lattice.c)+(conv_struc.lattice.c / conv_struc.lattice.a))/3
    struc_dict["V per Atoms"] = (conv_struc.volume / conv_struc.composition.num_atoms)

    return struc_dict


In [47]:
DT_df = pd.read_excel(r'Tester.xlsx')
DT_formulae = list(DT_df['Composition'])
DT_formulae = [Composition(formula).get_integer_formula_and_factor()[0] for formula in DT_df['Composition']]

materials_fields = [
    "material_id",
    "formula_pretty",
    "structure",
    "symmetry",
    "volume",
    "density",
]

with MPRester("Your Materials Project API") as mpr:
    docs = mpr.materials.search(formula=DT_formulae, fields=materials_fields)

amalgamated_dicts = []
for doc in tqdm(docs):
    try:
        amalgamated_dicts += [
            {
                "Composition": str(doc.formula_pretty),
                "Database IDs": doc.material_id,
                **get_struc_dict(doc.structure)
            }
        ]

    except IndexError:
        print(f"skipping composiiton {doc.formula_pretty} due to IndexError")
        continue

df = pd.DataFrame(amalgamated_dicts) #crystal structural feature set is created
df = df.round(3)

Retrieving MaterialsDoc documents:   0%|          | 0/17 [00:00<?, ?it/s]

100%|██████████| 17/17 [00:00<00:00, 154.25it/s]


In [48]:
class Vectorize_Formula:
    
    def __init__(self):
        elem_dict = pd.read_excel(r'elements_Debye.xlsx') # check the file name
        self.element_df = pd.DataFrame(elem_dict)
        self.element_df.set_index('Symbol',inplace=True)
        self.column_names = []
        for column_name in list(self.element_df.columns.values):
            for string in ['avg','diff','max','min']:
                self.column_names.append(f'{string}_{column_name}')

    def get_features(self, formula):
        try:
            fractional_composition = Composition(formula).fractional_composition.as_dict()
            avg_feature = np.zeros(len(self.element_df.columns))

            for key in fractional_composition:
                if key not in self.element_df.index:
                    print('The element:', key, 'from formula', formula,'is not currently supported in our database')
                    return np.full(len(self.element_df.columns)*4, np.nan)
                avg_feature += self.element_df.loc[key].values * fractional_composition[key]

            elements_in_formula = list(fractional_composition.keys())
            element_stats = self.element_df.loc[elements_in_formula]
            diff_feature = element_stats.max() - element_stats.min()
            max_feature = element_stats.max()
            min_feature = element_stats.min()

            features = []
            for i in range(len(self.element_df.columns)):
                features.extend([avg_feature[i], diff_feature.iloc[i], max_feature.iloc[i], min_feature.iloc[i]])
            return features
        except Exception as e:
            print(f"An error occurred while processing the formula {formula}: {e}")
            return np.full(len(self.element_df.columns) * 4, np.nan)

gf=Vectorize_Formula()

compositional_features=[]

for formula in df["Composition"]:
    compositional_features.append(gf.get_features(formula))

X = pd.DataFrame(compositional_features, columns = gf.column_names)
X = X.round(3)
composition_df=pd.concat([df, X], axis=1) # compositional features are added.


In [50]:
class Total_E_Calculation:
    
    def __init__(self):
        columns_to_read = ['Symbol', 'number of valence electrons', 'gilmor number of valence electron', 'outer shell electrons']
        elem_dict = pd.read_excel(r'elements_Debye.xlsx', usecols=columns_to_read)
        
        self.element_df = pd.DataFrame(elem_dict)
        self.element_df.set_index('Symbol', inplace=True)
        
        self.column_names = []
        for column_name in self.element_df.columns:
            self.column_names.append(f'total_{column_name}')

    def get_electron_densities(self, formula):        
        try:
            composition = Composition(formula)
            total_features = {col: 0 for col in self.column_names}
            
            for element, amount in composition.items():
                if element.symbol in self.element_df.index:
                    for col in self.element_df.columns:
                        total_features[f'total_{col}'] += self.element_df.loc[element.symbol, col] * amount
                else:
                    print(f"Warning: Element {element.symbol} not found in the database.")
            
            return total_features
        
        except Exception as e:
            print(f"Error processing formula {formula}: {str(e)}")
            return None
        
        
ef=Total_E_Calculation() # total number of each type of electrons to calculate four different electron densities

total_e=[]

for formula in composition_df["Composition"]:
    total_e.append(ef.get_electron_densities(formula))

Y = pd.DataFrame(total_e, columns = ef.column_names)

composition_df["Valence e per Reduced volume"]=Y["total_number of valence electrons"]/composition_df["Reduced Volume"]
composition_df["Valence e per V per Atom"]=Y["total_number of valence electrons"]/composition_df["V per Atoms"]
composition_df["Gilmor e per V per Atom"]=Y["total_gilmor number of valence electron"]/composition_df["V per Atoms"]
composition_df["Outer Shell e per V per Atom"]=Y["total_outer shell electrons"]/composition_df["V per Atoms"]

composition_df.round(3)
composition_df.to_excel('Tester_Feature_Set.xlsx', index=False)

### Debye Temperature Model with XGBoost

In [51]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import r2_score

In [52]:
DebyeDB=pd.read_excel('Debye_Temp_Training_Set.xlsx')
array = DebyeDB.values
X = array[:,2:153]
Y = array[:,1]
Compd = array[:,0]

best_model = XGBRegressor(tree_method='hist', device='cuda',  n_estimators=250, learning_rate=0.07,
                          max_depth=9, min_child_weight=7, subsample=0.7, base_score=0.6,
                          colsample_bytree=0.9, colsample_bylevel=0.8, colsample_bynode=1,
                          reg_alpha=0, reg_lambda=0.9
                          )
kf = KFold(n_splits=10, shuffle=True, random_state=42)
cv_results = cross_val_score(best_model, X, Y, cv=kf, scoring='r2')
print("\nCross-validation results for each fold:")
for i, score in enumerate(cv_results, 1):
    print(f"Fold {i}: R2 = {score:.4f}")
print(f"\nMean R2: {np.mean(cv_results):.4f}")
print(f"Standard Deviation of R2: {np.std(cv_results):.4f}")


Cross-validation results for each fold:
Fold 1: R2 = 0.9141
Fold 2: R2 = 0.9114
Fold 3: R2 = 0.9232
Fold 4: R2 = 0.9001
Fold 5: R2 = 0.9182
Fold 6: R2 = 0.9327
Fold 7: R2 = 0.9318
Fold 8: R2 = 0.9248
Fold 9: R2 = 0.9217
Fold 10: R2 = 0.9228

Mean R2: 0.9201
Standard Deviation of R2: 0.0092


In [53]:
best_model.fit(X,Y)
prediction = pd.read_excel(r'Tester_Feature_Set.xlsx')
a = prediction.values
b = a[:,2:152]
result = best_model.predict(b)
composition=prediction['Composition']
result=pd.DataFrame(result)
predicted=np.column_stack((composition,result))
predicted=pd.DataFrame(predicted)
predicted.round(3)
predicted.to_excel('Tester_Predicted_Debye.xlsx', index=False, header=("Composition","Predicted Debye"))